### Import necessary libraries

In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from shapely.geometry import Point
from sklearn.neighbors import KDTree
import os

### Loading Data

In [39]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel2.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/SampleSubmission.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel1.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Test.csv
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shx
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.dbf
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shp
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.dbf
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shx
/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shp


In [40]:
# Load Sentinel-1 and 2
s1 = pd.read_csv("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel1.csv").drop(columns=['date'])
s2 = pd.read_csv("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Sentinel2.csv").drop(columns=['date'])

In [41]:
s1.head()

,ID,VH,VV,orbit,polarization,rel_orbit,translated_lat,translated_lon
0,ID_AFQOFP,-21.479683,-16.633259,DESCENDING,"[VV, VH]",78.0,41.652292,72.144256
1,ID_AFQOFP,-24.769110,-15.943674,DESCENDING,"[VV, VH]",78.0,41.652289,72.144375
2,ID_AFQOFP,-25.370838,-15.185609,DESCENDING,"[VV, VH]",78.0,41.652286,72.144495
3,ID_AFQOFP,-24.134005,-16.351102,DESCENDING,"[VV, VH]",78.0,41.652283,72.144614
4,ID_AFQOFP,-20.654249,-16.792723,DESCENDING,"[VV, VH]",78.0,41.652280,72.144733


In [42]:
s2.head()

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,ID,cloud_pct,solar_azimuth,solar_zenith,translated_lat,translated_lon
0,2169,1820,1328,1610,1670,1985,2446,2628,2598,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935173,71.617062
1,2151,1770,1306,1586,1640,1961,2495,2691,2684,2732,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935171,71.617180
2,2169,1820,1456,1674,1808,1985,2446,2628,2486,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935085,71.616940
3,2169,1820,1284,1604,1658,1985,2446,2628,2658,2638,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935083,71.617059
4,2151,1770,1242,1522,1564,1961,2495,2691,2696,2732,ID_ZHZRHO,6.980395,139.093139,22.625533,40.935081,71.617177


### Data Preparation

In [43]:
# shape of sentinel 1 & 2
print('s1 shape:', s1.shape)
print('\n')
print('s2 shape:', s2.shape)

s1 shape: (1752570, 8)


s2 shape: (5610393, 16)


In [44]:
# Sentinel 1 & 2 information
s1.info()
print('-'*50)
s2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752570 entries, 0 to 1752569
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              object 
 1   VH              float64
 2   VV              float64
 3   orbit           object 
 4   polarization    object 
 5   rel_orbit       float64
 6   translated_lat  float64
 7   translated_lon  float64
dtypes: float64(5), object(3)
memory usage: 107.0+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5610393 entries, 0 to 5610392
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   B11             int64  
 1   B12             int64  
 2   B2              int64  
 3   B3              int64  
 4   B4              int64  
 5   B5              int64  
 6   B6              int64  
 7   B7              int64  
 8   B8              int64  
 9   B8A             int64  
 10  ID              object 
 11

In [45]:
# Check null values
print('s1')
print(s1.isna().sum())
print('-'*50)
print('s2')
print(s2.isna().sum())

s1
ID                0
VH                0
VV                0
orbit             0
polarization      0
rel_orbit         0
translated_lat    0
translated_lon    0
dtype: int64
--------------------------------------------------
s2
B11               0
B12               0
B2                0
B3                0
B4                0
B5                0
B6                0
B7                0
B8                0
B8A               0
ID                0
cloud_pct         0
solar_azimuth     0
solar_zenith      0
translated_lat    0
translated_lon    0
dtype: int64


### Extract Labelled GeoData

In [46]:
# Load shapefiles and extract labeled geodata
fergana_gdf = gpd.read_file("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Fergana_training_samples.shp")
orenburg_gdf = gpd.read_file("/kaggle/input/geoai-challenge-for-cropland-mapping-dry-dataset/Train/Orenburg_training_samples.shp")
train_gdf = pd.concat([fergana_gdf, orenburg_gdf])
train_gdf = train_gdf[['Cropland', 'geometry']]
train_gdf["lon"] = train_gdf.geometry.x
train_gdf["lat"] = train_gdf.geometry.y

In [47]:
# Match Sentinel2 ID points to nearest labeled point
tree = KDTree(train_gdf[["lat", "lon"]].values)
s2_points = s2.groupby("ID")[["translated_lat", "translated_lon"]].mean().reset_index()
dist, idx = tree.query(s2_points[["translated_lat", "translated_lon"]].values, k=1)
s2_points["label"] = train_gdf.iloc[idx.flatten()].Cropland.values
s2_labels = s2_points[["ID", "label"]]

In [48]:
# Aggregate function (excluding ID)
def aggregate_features(df, id_col="ID"):
    return df.drop(columns=["translated_lat", "translated_lon"], errors='ignore') \
             .groupby(id_col).agg(['mean', 'std', 'min', 'max']).reset_index()

# Aggregate train features
s1_feats = aggregate_features(s1[["ID", "VH", "VV"]])
s2_feats = aggregate_features(s2[["ID", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"]])

# Flatten multi-index columns
s1_feats.columns = ['_'.join(col).strip("_") for col in s1_feats.columns.values]
s2_feats.columns = ['_'.join(col).strip("_") for col in s2_feats.columns.values]

#### Merge training features and labels

In [49]:
# Merge training features and labels
train_df = s2_feats.merge(s1_feats, on="ID", how="outer").merge(s2_labels, on="ID", how="inner")
train_df = train_df.dropna()

In [50]:
train_df.head()

,ID,B2_mean,B2_std,B2_min,B2_max,B3_mean,B3_std,B3_min,B3_max,B4_mean,...,B12_max,VH_mean,VH_std,VH_min,VH_max,VV_mean,VV_std,VV_min,VV_max,label
0,ID_ABQOQT,2678.931516,2533.538818,749,16975,3087.956733,2446.428918,1015,16883,3313.985663,...,4938,-20.599840,3.788596,-48.104045,-11.837790,-10.241005,2.984731,-20.647057,-0.260766,0
1,ID_ADDROF,2180.467148,793.942887,314,10032,2551.151900,745.784220,684,9640,2725.154697,...,8095,-19.009345,3.215299,-37.340685,-9.947574,-9.077417,3.564367,-20.568176,6.495257,0
2,ID_AFIWZH,1760.097410,735.938649,76,6572,2015.750320,689.742879,342,6560,2055.006831,...,6238,-17.171517,2.445001,-33.776153,-10.090858,-7.539021,3.765559,-18.146540,4.695586,1
3,ID_AFQOFP,2459.410762,1700.451288,789,13704,2913.663884,1596.009589,1090,13088,3316.157296,...,7570,-24.251169,4.941170,-49.731128,-12.438228,-14.699856,3.231105,-25.396003,-2.288849,0
4,ID_AHRONV,2965.180911,3141.400874,439,10392,3160.444765,2983.807369,855,10456,3339.176704,...,7629,-25.955984,5.098076,-47.658529,-16.043004,-15.333656,3.817844,-26.721309,-5.322858,0


In [51]:
# Train-test split
X = train_df.drop(columns=["ID", "label"])
y = train_df["label"]
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)